# Finding guaranteed bounds on $\pi$ using standard floating-point arithmetic 

## By [David P. Sanders](http://sistemas.fciencias.unam.mx/~dsanders/)

### Department of Physics, Faculty of Sciences, National University of Mexico (UNAM)

In this notebook, we will calculate *guaranteed* (i.e., *validated*, or mathematically rigorous) bounds on $\pi$ using just floating-point arithmetic. This requires "directed rounding", i.e. the ability to control in which direction floating-point operations are rounded.

This is based on the book [*Validated Numerics*](http://press.princeton.edu/titles/9488.html) (Princeton, 2011) by [Warwick Tucker](http://www2.math.uu.se/~warwick/CAPA/warwick/warwick.html).

Consider the infinite series

$$ S := \sum_{n=1}^\infty \frac{1}{n^2},$$

whose exact value is [known](http://en.wikipedia.org/wiki/Basel_problem) to be $S = \frac{\pi^2}{6}$.
Thus, if finding guaranteed bounds on $S$ will give guaranteed bounds on $\pi$. 

The idea is to split $S$ up into two parts, $S = S_N + T_N$, where
$ S_N := \sum_{n=1}^N \frac{1}{n^2}$ contains the first $N$ terms, 
and $T_N := S - S_N = \sum_{n=N+1}^\infty \frac{1}{n^2}$ contains the rest (an infinite number of terms).

We will evalute $S_N$ numerically, and use the following analytical bound for $T_N$:

$$\frac{1}{N+1} \le T_N \le \frac{1}{N}$$.

This is obtained by approximating the sum in $T_N$ using integrals from below and above:

$$\int_{x=N+1}^\infty \frac{1}{x^2} dx \le T_N \le \int_{x=N}^\infty \frac{1}{x^2} dx.$$

$S_N$ may be calculated easily by summing either forwards or backwards:

In [ ]:
function forward_sum(N, T=Float64)
    total = zero(T)

    for i in 1:N
        total += one(T) / (i^2)
    end

    total
end

function reverse_sum(N, T=Float64)
    total = zero(T)

    for i in N:-1:1
        total += one(T) / (i^2)
    end
    
    total
end

To find *rigorous* bounds for $S_N$, we use "directed rounding", that is, we round downwards for the lower bound and  upwards for the upper bound:

In [ ]:
N = 10^6

lowerbound_S_N = 
    setrounding(Float64, RoundDown) do
        forward_sum(N)
    end

upperbound_S_N = 
    setrounding(Float64, RoundUp) do
        forward_sum(N)
    end

In [ ]:
lowerbound_S_N, upperbound_S_N

We incorporate the respective bound on $T_N$ to obtain the bounds on $S$, and hence on $\pi$:

In [ ]:
N = 10^6

lower_π =
    setrounding(Float64, RoundDown) do
        lower_bound = forward_sum(N) + 1/(N+1)
        sqrt(6 * lower_bound)
    end

upper_π = 
    setrounding(Float64, RoundUp) do
        upper_bound = forward_sum(N) + 1/N
        sqrt(6 * upper_bound)
    end

(lower_π, upper_π)

We may check that the true value of $\pi$ is indeed contained in the interval:

In [ ]:
lower_π < big(pi) < upper_π

Summing in the opposite direction turns out to give a more accurate answer:

In [ ]:
N = 10^6

lower_π =
    setrounding(Float64, RoundDown) do
        lower_bound = reverse_sum(N) + 1/(N+1)
        sqrt(6 * lower_bound)
    end

upper_π = 
    setrounding(Float64, RoundUp) do
        upper_bound = reverse_sum(N) + 1/N
        sqrt(6 * upper_bound)
    end

(lower_π, upper_π, lower_pi < big(pi) < upper_pi)

In principal, we could attain arbitrarily good precision with higher-precision `BigFloat`s, but the result is hampered by the slow convergence of the series.